In [1]:
!pip install gradio transformers python-docx pytesseract pillow PyMuPDF langdetect
!apt-get install -y tesseract-ocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 31.1 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=4f1ca0feca2e0973fb430eb2c0f7142efffe8c41bd642612275627f9706ac857
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [3]:
import gradio as gr
from transformers import pipeline
from PIL import Image
import pytesseract
from docx import Document
import fitz # PyMuPDF
from langdetect import detect

In [4]:
#English model
simplifier_en = pipeline("text2text-generation", model="t5-small")

#Translate to English first if language is Hindi or Bengali
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/707k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/791k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


In [28]:
#Extract text from file
def extract_text(file_obj):
    name = file_obj.name.lower()

    if name.endswith(".pdf"):
        doc = fitz.open(file_obj.name)
        return "\n".join(page.get_text() for page in doc)

    elif name.endswith(".docx"):
        doc = Document(file_obj.name)
        return "\n".join(p.text for p in doc.paragraphs)

    elif name.endswith(('.png', '.jpg', '.jpeg')):
        img = Image.open(file_obj.name)
        return pytesseract.image_to_string(img)

    elif name.endswith(".txt"):
        with open(file_obj.name, "r", encoding="utf-8") as f:
            return f.read()

    else:
        return ""

In [29]:
def smart_simplify(text):
    lang = detect(text)

    if lang in ["bn", "hi"]:
        translated = translator(text)[0]['translation_text']
    else:
        translated = text

    prompt = "simplify: " + translated
    result = simplifier_en(prompt, max_length=256, do_sample=False)
    return result[0]['generated_text']

In [30]:
def stylize_text(text):
    rules = {
        "important": "🔴 <b>IMPORTANT</b>",
        "warning": "⚠️ <b>WARNING</b>",
        "do not": "⛔️ <b>DO NOT</b>",
        "step": "🧩 <b>STEP</b>",
        "note": "📝 <b>NOTE</b>",
        "tip": "💡 <b>TIP</b>",
        "make sure": "✅ <b>MAKE SURE</b>",
    }

    for keyword, replacement in rules.items():
        text = text.replace(keyword, f"<span class='highlight'>{replacement}</span>")
    return text

In [37]:
def extract_text(file_obj):
    filepath = file_obj.name.lower()

    #PDF
    if filepath.endswith(".pdf"):
        with open(file_obj.name, "rb") as f:
            doc = fitz.open(stream=f.read(), filetype="pdf")
            return "\n".join([page.get_text() for page in doc])

    #DOCX
    elif filepath.endswith(".docx"):
        doc = Document(file_obj.name)
        return "\n".join([p.text for p in doc.paragraphs])

    #IMAGES (.jpg, .png, .jpeg)
    elif filepath.endswith((".jpg", ".png", ".jpeg")):
        img = Image.open(file_obj.name)
        return pytesseract.image_to_string(img)

    #TXT
    elif filepath.endswith(".txt"):
        with open(file_obj.name, "r", encoding="utf-8") as f:
            return f.read()

    return ""


In [44]:
def process(file):
    if file is None:
        return "<p>Please upload a valid file.</p>"

    try:
        raw_text = extract_text(file)
        if not raw_text or not raw_text.strip():
            return "<p>No readable text found or unsupported file type.</p>"

        chunks = [raw_text[i:i+400] for i in range(0, len(raw_text), 400)]
        simplified = ""
        for chunk in chunks:
            simplified += smart_simplify(chunk) + "\n"

        styled = stylize_text(simplified)

        styled_with_breaks = styled.replace('\n', '<br><br>')

        html_output = f"""
<div class="container">
    <h2>🧠 AI Simplified Output</h2>
    <div class="output-box">{styled_with_breaks}</div>
</div>
<style>
    .container {{
        font-family: 'Segoe UI', sans-serif;
        line-height: 1.7;
        color: #000;  /* fallback text color */
        padding: 20px;
    }}
    .output-box {{
        background-color: #fff2cc;  /* bright light yellow */
        border-left: 6px solid #fca311;
        padding: 15px;
        border-radius: 12px;
        font-size: 17px;
        color: #000 !important;  /* FORCE all text black */
    }}
    .output-box * {{
        color: #000 !important;  /* FORCE black inside all tags */
    }}
    .highlight {{
        color: #d62828 !important;  /* still highlight key words */
        font-weight: bold;
        font-size: 18px;
    }}
</style>
"""
        return html_output

    except Exception as e:
        return f"<p>Error: {str(e)}</p>"


In [45]:
#GradioUI
app = gr.Interface(
    fn=process,  # Ensure 'process' function is defined before this line
    inputs=gr.File(label="📤 Upload your file (PDF, DOCX, TXT, Image)"),
    outputs=gr.HTML(label="✨ Simplified Result"),
    title="📚 AI Text Simplifier with Language Detection",
    description="Upload a document in English, Hindi, or Bengali. The app will auto-detect the language, simplify the text, and highlight important info with icons.",
)

app.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7c0e0e89a0a29718dd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
